In [ ]:
import pandas as pd
import request
import json
import numpy as np
import datetime

In [ ]:
# your credentials
import ig_credentials as pw

In [ ]:
# create an Object container
APIs = {}

In [ ]:
# DO NOT RUN THIS TOO OFTEN! YOU WILL GET BLOCKED! THEN YOU HAVE TO MOVE!

import instagram_private_api
APIs.update({"instagram" : instagram_private_api.Client(pw.IG_USER, pw.IG_PASS)})

In [ ]:
# enter ig account name that you want to scrape
ig_name = 'dovechocolate'

In [ ]:
# define a function to get instagram ID
def ig_name_to_id(ig_name, Client):
    return Client.username_info(ig_name)['user']['pk']

In [ ]:
ig_id = ig_name_to_id(ig_name, APIs['instagram'])

In [ ]:
# the actual scrape process

MAX_LEN = 1100 # enter number of posts you want to scrape

response = APIs['instagram'].user_feed(ig_id)
media = response['items']
while len(media) < MAX_LEN and 'next_max_id' in response.keys():
    response = APIs['instagram'].user_feed(ig_id, max_id=response['next_max_id'])
    media.extend(response['items'])

ig_data = pd.io.json.json_normalize(media)

In [ ]:
len(ig_data)

In [ ]:
# select the columns you need from all the columns provided
columns_to_keep =  [
    'id',
    'media_type',
    'caption.text',
    'code',
    'like_count',
    'comment_count',
    'view_count',
    'video_duration',
    'carousel_media_count',
    'view_count',
    'caption.created_at_utc',
    'image_versions2.candidates']

ig_data_processed = ig_data[columns_to_keep]

In [ ]:
# notes to read the result:
# media_type = 1 >>> Photo
# media_type = 2 >>> Video
# media_type = 8 >>> Carousel/Albumx

In [ ]:
ig_data_processed[ig_data_processed['caption.created_at_utc'].isnull()] = 0

In [ ]:
# clean up time format
caption_created_time = []

i = 0
for i in ig_data_processed['caption.created_at_utc']:
    value = datetime.datetime.fromtimestamp(int(i)).strftime('%Y-%m-%d %H:%M:%S')
    caption_created_time.append(value)
    i = i + 1

ig_data_processed['caption_created_time'] = caption_created_time 

In [ ]:
# reset index and save the data
ig_data_processed = ig_data_processed.reset_index(drop=True)
ig_data_processed.to_pickle(ig_name + '_ig.pkl')